# Assignment 2A

In [15]:
# Importing Necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp

In [16]:
# Importing Data
returns = pd.read_excel('Data_2A_2023.xlsx',index_col='Date') / 100
display(returns.describe())

,AAPL,DIS,GE,GS,MSFT
count,5787.000000,5787.000000,5787.000000,5787.000000,5787.000000
mean,0.001213,0.000422,0.000059,0.000540,0.000513
std,0.025148,0.019490,0.021196,0.023153,0.019388
min,-0.518691,-0.183630,-0.151592,-0.189596,-0.155978
25%,-0.010370,-0.008772,-0.008820,-0.010168,-0.008268
50%,0.000872,0.000289,0.000000,0.000307,0.000347
75%,0.013145,0.009135,0.008968,0.011173,0.009376
max,0.139050,0.159722,0.197031,0.264678,0.195652


In [17]:
# Train, Test Split
train_set = returns[:"2015"]
test_set = returns["2016":]

# Compute the first and second Moments
average_return = train_set.mean()
covariance_matrix = train_set.cov(ddof=0) # divide over T rather than T-1 as indicated in assignment

In [18]:
def optimize_portfolio(expected_returns, cov_matrix, required_return):
    """
    Optimizes the vector of portfolio weights according to the first optimization problem in Assignment 2A
    """
    # Define Decision variables
    num_assets = len(expected_returns)
    weights = cp.Variable(num_assets, nonneg=False)
    
    # Compute Performance Metrics
    portfolio_return = cp.matmul(weights, expected_returns)
    portfolio_variance = cp.quad_form(weights, cov_matrix)
    
    # Set up formulation
    constraints = [cp.sum(weights) == 1, portfolio_return == required_return]
    objective = cp.Minimize(portfolio_variance)
    optimization_problem = cp.Problem(objective, constraints)
    
    # Solve Optimization Problem
    optimization_problem.solve()
    
    # Return Solution
    if optimization_problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        return weights.value
    else:
        print('No solution found')
        return None 

In [19]:
average_positive_return = average_return[average_return > 0].mean() # Determine the Target Return

# Determine the Optimal Weights
optimized_weights = optimize_portfolio(average_return, covariance_matrix, average_positive_return)

# Print Results
optimized_weights_df = pd.DataFrame(optimized_weights, index = returns.columns, columns=['Optimized Weights'])
display(optimized_weights_df)

,Optimized Weights
AAPL,0.196071
DIS,0.338601
GE,0.195632
GS,0.036503
MSFT,0.233194


In [20]:
# Calculating Performance Metrics
def calculate_performance(portfolio_weights, average_returns, covariance_matrix):
    """ 
    Calculate the annualised performance metrics of the portfolio based on weights and mean and cov estimates.
    """
    annual_portfolio_return = np.dot(portfolio_weights, average_returns) * 252
    annual_portfolio_variance = np.dot(portfolio_weights, np.dot(covariance_matrix, portfolio_weights)) * 252
    return annual_portfolio_return, annual_portfolio_variance

def compute_portfolio_returns(returns_df, initial_weights):
    """
    Computes the portfolio returns over time, accounting for drift in weights over time.
    """
    current_weights = initial_weights.copy()
    dates = returns_df.index
    portfolio_returns = []  

    for current_date in dates:
        # Calculate the return for the current date by dot product of returns and current weights
        portfolio_returns.append(np.dot(returns_df.loc[current_date], current_weights))
        
        # Recompute drifted weights based on days returns
        drifted_weights = current_weights * (1 + returns_df.loc[current_date])
        total_value = np.sum(drifted_weights)
        current_weights = drifted_weights / total_value
        
    portfolio_returns_df = pd.Series(portfolio_returns, index=dates)
    
    return portfolio_returns_df

In [21]:
# Compute the Mean-Variance estimates of the optimised portfolio
annual_return, annual_variance = calculate_performance(optimized_weights, average_return, covariance_matrix)

# Compute the out-of-sample performance for the mean-varience portfolio with drift
mv_portfolio_returns = compute_portfolio_returns(test_set, optimized_weights)

# Compute the out-of-sample performance for the mean-varience portfolio with constant rebalancing
oos_return, oos_variance = calculate_performance(optimized_weights, test_set.mean(),test_set.cov())

results = pd.DataFrame({
    'Value': [f'{annual_return:.2%}', f'{annual_variance:.3f}', 
              f'{mv_portfolio_returns.mean() * 252:.2%}', f'{mv_portfolio_returns.var() * 252:.3f}', 
              f'{oos_return:.2%}', f'{oos_variance:.3f}']
}, index=['Predicted Annual Return', 'Predicted Annual Variance', 
          'Annualised Return (with drift)', 'Annualised Variance (with drift)', 
          'Annualised Return (constant rebalancing)', 'Annualised Variance (constant rebalancing)'])
display(results)

,Value
Predicted Annual Return,14.03%
Predicted Annual Variance,0.066
Annualised Return (with drift),16.82%
Annualised Variance (with drift),0.059
Annualised Return (constant rebalancing),11.95%
Annualised Variance (constant rebalancing),0.057


In [22]:
# Minimum Variance Optimization
def minimize_variance(covariance_matrix):
    """
    Optimizes the vector of portfolio weights according to the second optimization problem in Assignment 2A
    """
    # Define Decision Variables
    num_assets= covariance_matrix.shape[0]
    weights = cp.Variable(num_assets, nonneg=False)
    
    # Define Performance Metrics
    portfolio_variance = cp.quad_form(weights, covariance_matrix)
    
    # Define Formulation
    constraints = [cp.sum(weights) == 1]
    objective = cp.Minimize(portfolio_variance)
    problem = cp.Problem(objective, constraints)
    
    # Solve Optimization Problem
    problem.solve()
    
    # Return solution
    if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        return weights.value
    else:
        print('No solution found')
        return None 

# Determine Minimum Variance Weights
minimum_variance_weights = minimize_variance(covariance_matrix)
minimum_variance_weights_df = pd.DataFrame(minimum_variance_weights, index=returns.columns, columns=['Minimum Variance Weights'])
display(minimum_variance_weights_df)

,Minimum Variance Weights
AAPL,0.090358
DIS,0.288803
GE,0.293817
GS,0.020338
MSFT,0.306683


In [23]:
# Compute the Mean-Variance estimates of the minimum variance portfolio
min_var_annual_return, min_var_annual_variance = calculate_performance(minimum_variance_weights, average_return, covariance_matrix)

# Compute the out-of-sample performance for the minimum variance portfolio with drift
min_var_portfolio_returns = compute_portfolio_returns(test_set, minimum_variance_weights)

# Compute the out-of-sample performance for the minimum variance portfolio with constant rebalancing
min_var_oos_return, min_var_oos_variance = calculate_performance(minimum_variance_weights, test_set.mean(), test_set.cov())

# Create a DataFrame for the results of the minimum variance portfolio
min_var_results = pd.DataFrame({
    'Value': [f'{min_var_annual_return:.2%}', f'{min_var_annual_variance:.3f}', 
              f'{min_var_portfolio_returns.mean() * 252:.2%}', f'{min_var_portfolio_returns.var() * 252:.3f}', 
              f'{min_var_oos_return:.2%}', f'{min_var_oos_variance:.3f}']
}, index=['Predicted Annual Return', 'Predicted Annual Variance', 
          'Annualised Return (with drift)', 'Annualised Variance (with drift)', 
          'Annualised Return (constant rebalancing)', 'Annualised Variance (constant rebalancing)'])

# Display the results for the minimum variance portfolio
display(min_var_results)

,Value
Predicted Annual Return,10.79%
Predicted Annual Variance,0.064
Annualised Return (with drift),15.34%
Annualised Variance (with drift),0.059
Annualised Return (constant rebalancing),9.88%
Annualised Variance (constant rebalancing),0.060
